<a href="https://colab.research.google.com/github/Ines-Abdelaziz/Sentiment-Analysis-via-RNN/blob/main/TP_DeepLearningRNN_BDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense,Bidirectional
from tensorflow.keras.models import Sequential

In [ ]:
#Loading and preprocessing the data
df = pd.read_csv('extracted_data/sentiment140.csv', encoding='ISO-8859-1', header=None)
df = df.iloc[:, [0, 5]]
df.columns = ['label', 'text']
df

,label,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


- We only use the first(sentiment) and sixth(text) column of the dataset.
- we rename them as label and text


In [ ]:

df['text'] = df['text'].apply(lambda x: x.lower())
df['text'] = df['text'].str.replace('[^\w\s]', '')
df['text'] = df['text'].str.replace('\d+', '')
df['text'] = df['text'].str.strip()

<ipython-input-28-087f5e3bc282>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]', '')
<ipython-input-28-087f5e3bc282>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('\d+', '')


- we  convert all the tweet texts to lowercase
- we remove all non-alphanumeric characters and numbers
- we remove any leading and trailing white spaces

In [ ]:
#labeling the data
df['label'] = df['label'].replace({0: 'negative', 4: 'positive'})
#Splitting the data
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
val_data = df[train_size:]
# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data['text'])

# Padding
train_sequences = tokenizer.texts_to_sequences(train_data['text'])
train_padded = pad_sequences(train_sequences, maxlen=50, padding='post', truncating='post')
val_sequences = tokenizer.texts_to_sequences(val_data['text'])
val_padded = pad_sequences(val_sequences, maxlen=50, padding='post', truncating='post')

- we replace numeric values of the label
- we split the data 80% to train and 20% for testing
- We tokenize and padd the data


In [ ]:
#model architecture
model = Sequential([
    Embedding(input_dim=10000, output_dim=32, input_length=50),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(2, activation='softmax')
])

#Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#Summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 32)            320000    
                                                                 
 lstm_1 (LSTM)               (None, 64)                24832     
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 344,962
Trainable params: 344,962
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Training the model
history = model.fit(train_padded, tf.keras.utils.to_categorical(train_data['label'].map({'negative': 0, 'positive': 1})),
                    validation_data=(val_padded, tf.keras.utils.to_categorical(val_data['label'].map({'negative': 0, 'positive': 1}))),
                    epochs=10, batch_size=128)

# Evaluating the model
val_loss, val_acc = model.evaluate(val_padded, tf.keras.utils.to_categorical(val_data['label'].map({'negative': 0, 'positive': 1})))
print('Validation loss:', val_loss)
print('Validation accuracy:', val_acc)

Epoch 1/10
10000/10000 [==============================] - 740s 74ms/step - loss: 0.4214 - accuracy: 0.8068 - val_loss: 0.5789 - val_accuracy: 0.7277
Epoch 2/10
10000/10000 [==============================] - 733s 73ms/step - loss: 0.3849 - accuracy: 0.8260 - val_loss: 0.6659 - val_accuracy: 0.6517
Epoch 3/10
10000/10000 [==============================] - 742s 74ms/step - loss: 0.3728 - accuracy: 0.8322 - val_loss: 0.5718 - val_accuracy: 0.7334
Epoch 4/10
10000/10000 [==============================] - 735s 73ms/step - loss: 0.3636 - accuracy: 0.8372 - val_loss: 0.5663 - val_accuracy: 0.7389
Epoch 5/10
10000/10000 [==============================] - 733s 73ms/step - loss: 0.3565 - accuracy: 0.8412 - val_loss: 0.5801 - val_accuracy: 0.7238
Epoch 6/10
10000/10000 [==============================] - 747s 75ms/step - loss: 0.3510 - accuracy: 0.8439 - val_loss: 0.5802 - val_accuracy: 0.7236
Epoch 7/10
10000/10000 [==============================] - 738s 74ms/step - loss: 0.3460 - accuracy: 0.8469